In [1]:
import nba_py
import datetime
import time
import nba_py.game
import nba_py.player
import nba_py.team
import pandas as pd
import numpy as np
import functools

today = datetime.date.today()
tomorrow = today + datetime.timedelta(days = 1)
someday = datetime.date(2017, 1, 1)

In [2]:
def get_games(date):
    '''
    return a pandas.Series of all the games available on this day.
    '''
    return nba_py.Scoreboard(month = date.month, 
                             day = date.day, 
                             year = date.year, 
                             league_id = '00', 
                             offset = 0).available()['GAME_ID']

games = get_games(someday)
games

0    0021600510
1    0021600509
2    0021600508
3    0021600511
4    0021600512
Name: GAME_ID, dtype: object

In [3]:
def get_teams(games):
    '''
    return a pandas.DataFrame including every games' home team id and away team id.
    '''
    teams = [nba_py.game.BoxscoreSummary(g).game_summary().loc[0, ['HOME_TEAM_ID', 'VISITOR_TEAM_ID']] 
             for g in games]
    return pd.DataFrame(teams, index = [g for g in games])

teams = get_teams(games)
teams

,HOME_TEAM_ID,VISITOR_TEAM_ID
0021600510,1610612754,1610612753
0021600509,1610612748,1610612765
0021600508,1610612737,1610612759
0021600511,1610612750,1610612757
0021600512,1610612747,1610612761


In [4]:
def get_players(teams):
    '''
    return a pandas.DataFrame of all the teams' players.
    teams must be a pandas.DataFrame created by the function get_teams().
    '''
    player_list = nba_py.player.PlayerList().info()
    all_teams = teams['HOME_TEAM_ID'].append(teams['VISITOR_TEAM_ID'])
    players = functools.reduce(pd.Series.append, 
                                [nba_py.team.TeamPlayers(team).season_totals()['PLAYER_ID'] 
                                 for team in all_teams])
    players = players.to_frame()
    players['TEAM_ID'] = [int(player_list[player_list['PERSON_ID'] == p]['TEAM_ID']) 
                          for p in players['PLAYER_ID']]
    players = players[players['TEAM_ID'].isin(all_teams.values)]
    players['AGAINST_TEAM_ID'] = [int(teams[teams['HOME_TEAM_ID'] == t]['VISITOR_TEAM_ID'] 
                              if t in teams['HOME_TEAM_ID'].values 
                              else teams[teams['VISITOR_TEAM_ID'] == t]['HOME_TEAM_ID']) 
                          for t in players['TEAM_ID']]
    return players
    

get_players(teams)

,PLAYER_ID,TEAM_ID,AGAINST_TEAM_ID
0,201166,1610612754,1610612753
1,2744,1610612754,1610612753
2,101139,1610612754,1610612753
3,1627777,1610612754,1610612753
4,203922,1610612754,1610612753
5,201952,1610612754,1610612753
6,1626202,1610612754,1610612753
7,202338,1610612754,1610612753
8,202362,1610612754,1610612753
9,202730,1610612754,1610612753


In [5]:
nba_py.player.PlayerLastNGamesSplits('201939').last10().columns

Index(['GROUP_SET', 'GROUP_VALUE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD',
       'PTS', 'PLUS_MINUS', 'DD2', 'TD3', 'GP_RANK', 'W_RANK', 'L_RANK',
       'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK',
       'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK',
       'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK',
       'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK',
       'PTS_RANK', 'PLUS_MINUS_RANK', 'DD2_RANK', 'TD3_RANK', 'CFID',
       'CFPARAMS'],
      dtype='object')

In [6]:
play_stats_cols = ['GROUP_VALUE', 'GROUP_SET', 'GP', 
                   'MIN', 'PTS', 'AST', 'OREB', 'DREB', 'STL', 'BLK', 'TOV', 
                   'FGM', 'FGA', 'FG3M']

In [7]:
stats_201939 = nba_py.player.PlayerLastNGamesSplits('201939', per_mode = 'Per36').last10()[play_stats_cols]
stats_201939

,GROUP_VALUE,GROUP_SET,GP,MIN,PTS,AST,OREB,DREB,STL,BLK,TOV,FGM,FGA,FG3M
0,2016-17,Last 10 Games,10,331.868333,30.8,9.3,0.9,4.1,2.6,0.4,3.5,10.5,21.0,5.6


In [70]:
nba_py.player.PlayerLastNGamesSplits('201939', location = 'Home').last20()[play_stats_cols].iloc[0]

GROUP_VALUE          2016-17
GROUP_SET      Last 20 Games
GP                        11
MIN                     32.6
PTS                       26
AST                      7.4
OREB                     0.9
DREB                     3.1
STL                      2.4
BLK                      0.4
TOV                      3.2
FGM                      8.6
FGA                     17.6
FG3M                     4.7
Name: 0, dtype: object

In [9]:
def cal_score(stats):
    play_stats_cols = ['GROUP_VALUE', 'GROUP_SET', 'GP', 
                       'MIN', 'PTS', 'AST', 'OREB', 'DREB', 'STL', 'BLK', 'TOV', 
                       'FGM', 'FGA', 'FG3M']
    stats = stats[play_stats_cols].iloc[0]
    score = stats['PTS'] * 1 + stats['AST'] * 1.5 + \
    stats['OREB'] * 1 + stats['DREB'] * 0.7 + \
    stats['STL'] * 2 + stats['BLK'] * 1.8 + stats['TOV'] * -1 + \
    stats['FGM'] * 0.4 + (stats['FGA'] - stats['FGM']) * -1 + stats['FG3M'] * 0.5
    return score

round(float(cal_score(stats_201939)), 2)

47.44

In [12]:
game_date = nba_py.game.BoxscoreSummary('0021601229').game_info().loc[0, 'GAME_DATE']
game_date

'WEDNESDAY, APRIL 12, 2017'

In [13]:
datetime_object = time.strptime(game_date, '%A, %B %d, %Y')
datetime_object

time.struct_time(tm_year=2017, tm_mon=4, tm_mday=12, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=2, tm_yday=102, tm_isdst=-1)

In [14]:
dt = datetime.datetime.fromtimestamp(time.mktime(datetime_object))
dt

datetime.datetime(2017, 4, 12, 0, 0)

In [16]:
str(dt)

'2017-04-12 00:00:00'

In [132]:
game_index = game_logs[game_logs['Game_ID']  <= '0021601136'].index[:5]

In [134]:
game_logs.loc[game_index]

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
5,22016,201939,0021601136,"MAR 31, 2017",GSW vs. HOU,W,35,8,18,0.444,...,4,7,5,2,0,2,4,24,14,1
6,22016,201939,0021601118,"MAR 29, 2017",GSW @ SAS,W,35,9,20,0.450,...,3,4,11,1,0,2,5,29,6,1
7,22016,201939,0021601108,"MAR 28, 2017",GSW @ HOU,W,33,11,22,0.500,...,8,10,7,2,0,6,5,32,9,1
8,22016,201939,0021601094,"MAR 26, 2017",GSW vs. MEM,W,36,8,18,0.444,...,1,1,11,4,0,5,1,21,8,1
9,22016,201939,0021601081,"MAR 24, 2017",GSW vs. SAC,W,33,9,18,0.500,...,6,7,12,6,0,3,0,27,28,1


In [133]:
game_index

Int64Index([5, 6, 7, 8, 9], dtype='int64')

In [182]:
def get_last_n_game_logs(player_ID, game_ID, n):
    '''
    retrun a pandas.DataFrame of a player's last n game logs before a specific game.
    '''
    game_logs = nba_py.player.PlayerGameLogs(player_ID).info()
    game_index = game_logs[game_logs['Game_ID']  < game_ID].index[:n]
    return game_logs.loc[game_index, ['MIN', 'PTS', 'AST', 'OREB', 'DREB', 'STL', 'BLK',
                                      'TOV', 'FGM', 'FGA', 'FG3M']]

last_n_game_201939 = get_last_n_game_logs('201939', '0021600437', 20)
last_n_game_201939

,MIN,PTS,AST,OREB,DREB,STL,BLK,TOV,FGM,FGA,FG3M
50,30,25,4,0,3,3,0,0,8,18,4
51,27,19,6,0,3,0,0,3,6,11,5
52,34,8,8,1,9,1,0,3,3,14,2
53,34,30,7,2,2,1,0,2,11,23,5
54,32,22,9,1,3,1,0,0,6,13,2
55,31,17,3,2,6,0,0,4,4,14,3
56,36,26,2,0,5,2,0,3,7,16,3
57,33,19,6,0,4,7,0,3,7,16,0
58,29,13,11,0,4,2,1,0,4,10,2
59,30,31,1,0,2,1,0,3,10,15,5


In [175]:
def get_score_36(game_logs):
    '''
    return the Fantasy score of the given game logs(pandas.DataFrame).
    '''
    convert_to_36 = lambda x: x[['PTS', 'AST', 'OREB', 'DREB', 'STL', 'BLK', 
                                 'TOV', 'FGM', 'FGA', 'FG3M']] * 36 / x['MIN']
    stats = game_logs.apply(convert_to_36, axis = 1).mean()
    score = stats['PTS'] * 1 + stats['AST'] * 1.5 + \
    stats['OREB'] * 1 + stats['DREB'] * 0.7 + \
    stats['STL'] * 2 + stats['BLK'] * 1.8 + stats['TOV'] * -1 + \
    stats['FGM'] * 0.4 + (stats['FGA'] - stats['FGM']) * -1 + stats['FG3M'] * 0.5
    return score

get_score_36(last_n_game_201939)

37.260176519625048

In [186]:
def predict_score_36(player_ID, game_ID):
    ma_20 = get_score_36(get_last_n_game_logs(player_ID, game_ID, 20))
    ma_10 = get_score_36(get_last_n_game_logs(player_ID, game_ID, 10))
    ma_5 = get_score_36(get_last_n_game_logs(player_ID, game_ID, 5))
    return [round(float(ma_20), 2), round(float(ma_10), 2), round(float(ma_5), 2)]

predict_score_36('201939', '0021601211')

[40.67, 46.93, 49.66]

In [178]:
nba_py.player.PlayerGameLogs('201939').info()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22016,201939,0021601229,"APR 12, 2017",GSW vs. LAL,W,28,6,17,0.353,...,5,5,8,2,0,6,1,20,27,1
1,22016,201939,0021601211,"APR 10, 2017",GSW vs. UTA,L,30,9,16,0.563,...,3,4,4,1,0,1,1,28,9,1
2,22016,201939,0021601171,"APR 05, 2017",GSW @ PHX,W,35,15,25,0.600,...,4,4,11,2,2,3,0,42,25,1
3,22016,201939,0021601162,"APR 04, 2017",GSW vs. MIN,W,30,7,18,0.389,...,3,3,9,3,2,3,2,19,13,1
4,22016,201939,0021601152,"APR 02, 2017",GSW vs. WAS,W,36,15,22,0.682,...,1,1,8,1,0,1,3,42,26,1
5,22016,201939,0021601136,"MAR 31, 2017",GSW vs. HOU,W,35,8,18,0.444,...,4,7,5,2,0,2,4,24,14,1
6,22016,201939,0021601118,"MAR 29, 2017",GSW @ SAS,W,35,9,20,0.450,...,3,4,11,1,0,2,5,29,6,1
7,22016,201939,0021601108,"MAR 28, 2017",GSW @ HOU,W,33,11,22,0.500,...,8,10,7,2,0,6,5,32,9,1
8,22016,201939,0021601094,"MAR 26, 2017",GSW vs. MEM,W,36,8,18,0.444,...,1,1,11,4,0,5,1,21,8,1
9,22016,201939,0021601081,"MAR 24, 2017",GSW vs. SAC,W,33,9,18,0.500,...,6,7,12,6,0,3,0,27,28,1
